In [1]:
import json

exp_name = 'memory_core_1'
test_data = json.load(open(f'output/{exp_name}/predicted_type.json', 'r'))
template_and_replacements_examples = json.load(open('data/example/template_and_replacements.json', 'r'))

In [2]:
simple_question_type_to_templates = {
    "compare": ["(compare (GROUP_COUNT x1) number)", "(compare (GROUP_COUNT x1) x2)", "(compare (GROUP_SUM (GROUP_COUNT x1) (GROUP_COUNT x2)) (OR x3 x4))", "(compare (GROUP_SUM (GROUP_COUNT x1) (GROUP_COUNT x2)) number)"],
    "compare_and_count": ["(COUNT (compare (GROUP_COUNT x1) number))", "(COUNT (compare (GROUP_COUNT x1) x2))", "(COUNT (compare (GROUP_SUM (GROUP_COUNT x1) (GROUP_COUNT x2)) (OR x3 x4)))", "(COUNT (compare (GROUP_SUM (GROUP_COUNT x1) (GROUP_COUNT x2)) number))"],
    "count": ["(COUNT (DISTINCT (OR x1 x2)))", "(COUNT (DISTINCT x1))", "(COUNT (GROUP_COUNT x1))", "(COUNT (GROUP_SUM (GROUP_COUNT x1) (GROUP_COUNT x2)))", "(COUNT x1)"],
    "optimize": ["(optimize (GROUP_COUNT x1))", "(optimize (GROUP_SUM (GROUP_COUNT x1) (GROUP_COUNT x2)))"],
    "simple": ["(DIFF x1 x2)", "(DISTINCT x1)", "(GROUP_COUNT x1)", "(GROUP_SUM (GROUP_COUNT x1) (GROUP_COUNT x2))", "(OR x1 x2)", "x1"],
    "verify": ["(ALL x1 x2 x3 x4 x5 x6 x7 x8 x9 x10)", "(ALL x1 x2 x3 x4 x5 x6 x7 x8 x9)", "(ALL x1 x2 x3 x4 x5 x6 x7 x8)", "(ALL x1 x2 x3 x4 x5 x6 x7)", "(ALL x1 x2 x3 x4 x5 x6)", "(ALL x1 x2 x3 x4 x5)", "(ALL x1 x2 x3 x4)", "(ALL x1 x2 x3)", "(ALL x1 x2)", "(ALL x1)"]
}

In [3]:
examples = {}
for simple_question_type in template_and_replacements_examples:
    examples[simple_question_type] = []
    for qa in template_and_replacements_examples[simple_question_type]:
        examples[simple_question_type].append({'role': 'user',
        'content': "question: " + qa['coreference_resolved_question']
         + "\ncandidate_templates: " + ','.join(simple_question_type_to_templates[simple_question_type])
         + "\ncandidate_s_expression_cores: " + '\u001F'.join(qa['s_expression_cores_fn'])})
        replacements = {"variables": {var: qa["s_expression_cores_fn"][qa['replacements']['variables'][var]] for var in qa['replacements']['variables']}, "constants": qa['replacements']['constants'], "functions": qa['replacements']['functions']}
        examples[simple_question_type].append({'role': 'assistant', 'content': json.dumps({'template': qa['template'], 'replacements': replacements})})
examples

{'verify': [{'role': 'user',
   'content': 'question: Output: Do Didrik Ficks gränd have Västerlånggatan and Stora Nygatan as terminus?\ncandidate_templates: (ALL x1 x2 x3 x4 x5 x6 x7 x8 x9 x10),(ALL x1 x2 x3 x4 x5 x6 x7 x8 x9),(ALL x1 x2 x3 x4 x5 x6 x7 x8),(ALL x1 x2 x3 x4 x5 x6 x7),(ALL x1 x2 x3 x4 x5 x6),(ALL x1 x2 x3 x4 x5),(ALL x1 x2 x3 x4),(ALL x1 x2 x3),(ALL x1 x2),(ALL x1)\ncandidate_s_expression_cores: (IS_TRUE Didrik_Ficks_gränd terminus Västerlånggatan)\x1f(IS_TRUE Didrik_Ficks_gränd terminus Stora_Nygatan)'},
  {'role': 'assistant',
   'content': '{"template": "(ALL x1 x2)", "replacements": {"variables": {"x2": "(IS_TRUE Didrik_Ficks_gr\\u00e4nd terminus Stora_Nygatan)", "x1": "(IS_TRUE Didrik_Ficks_gr\\u00e4nd terminus V\\u00e4sterl\\u00e5nggatan)"}, "constants": {}, "functions": {}}}'},
  {'role': 'user',
   'content': 'question: Are Canada next to the border of Prince Edward Island and Greenland?\ncandidate_templates: (ALL x1 x2 x3 x4 x5 x6 x7 x8 x9 x10),(ALL x1 x2 x3 x4

In [4]:
def fix_json(s):
    return s + '}' * (s.count('{') - s.count('}'))

def fix_template(template):
    return template + ')' * (template.count('(') - template.count(')'))

def is_close(expr):
    stack = 0
    for char in expr:
        if char == '(':
            stack += 1
        elif char == ')':
            if stack == 0:
                return False
            stack -= 1
    return stack == 0

def fix_core(core):
    tokens = [token.replace('(', '').replace(')', '') for token in core.split()]
    index = 0
    def parse_core():
        nonlocal index
        func_list = ['JOIN', 'R', 'AND', 'VALUES', 'IS_TRUE']
        token = tokens[index]
        if token in func_list:
            index += 1
            args = []
            if token == 'IS_TRUE':
                for _ in range(3):
                    args.append(parse_core())
            elif token == 'JOIN':
                for _ in range(2):
                    args.append(parse_core())
            elif token == 'R':
                args.append(parse_core())
            elif token == 'AND':
                while index < len(tokens):
                    args.append(parse_core())
            else:
                while index < len(tokens) and tokens[index] not in func_list:
                    args.append(parse_core())
            return f'({token} {' '.join(args)})'
        else:
            value = token
            index += 1
            return value
    try:
        fixed_core = parse_core()
    except:
        fixed_core = core
    return fixed_core

In [ ]:
import re
from utils.llm_call import LLM_Call

def concat_input(qa):
    return "question:" + qa['coreference_resolved_question'] \
        + "\ncandidate_templates:" + ','.join(simple_question_type_to_templates[qa['predicted_simple_question_type']]) \
        + "\ncandidate_s_expression_cores:" + '\u001F'.join(qa['calibrated_cores_fn'])

API_KEYS = ['sk-54964e5c3b8c4998a74f7d3e35b618ac', 'sk-785362add9834d1da5907f4621dc90a8']
API_URL = 'https://api.deepseek.com'
API_MODEL = 'deepseek-chat'

api_pool = [(k, API_URL, API_MODEL) for k in API_KEYS]

LLM = LLM_Call(api_pool=api_pool, openai_params = {'temperature': 0, 'max_tokens': 512})

batch_size = 20
batches = [test_data[i:i + batch_size] for i in range(0, len(test_data), batch_size)]

prompt = """
    # S-expression Generation Task

    ## Core Task
    Select an appropriate template from candidate_templates and replace placeholders in the template with numbers from the question, specified functions, or expressions from candidate_s_expression_cores to form a final expression that matches the question's semantics. Output the replacements made.

    ## Key Principles
    1. Semantics First: Replacements must accurately express the question's logic
    2. Strict Replacement: Only replace placeholders while preserving all other template structures
    3. Complete Coverage: Ensure all placeholders are replaced

    ## Operator Semantics
    | Operator  | Description                     | Example                      |
    |-----------|--------------------------------|-----------------------------|
    | OR        | Set union (A ∪ B)              | (OR a b) → union of a and b |
    | DIFF      | Set difference (A - B)         | (DIFF a b) → elements in a but not b |
    | DISTINCT  | Deduplication                  | (DISTINCT a) → deduplicated a |
    | GROUP_COUNT| Group counting statistics     | (GROUP_COUNT a) → count of a |
    | GROUP_SUM | Sum of grouped statistics     | (GROUP_SUM a b) → sum of a and b's stats |
    | ALL       | Universal quantifier          | (ALL a b) → a∧b            |

    ## Replacement Rules
    1. Variable placeholder replacement:
    - Must select from candidate_s_expression_cores
    - Selected core text must remain unchanged
    - Determine replacement order based on question semantics
    - Different x should use different cores

    2. Constant placeholder replacement:
    - number → Explicit number from the question

    3. Function placeholder replacement:
    - compare → Select based on question semantics:
        - LT(less than) / LE(less than or equal to) / EQ(equal to) / GE(greater than or equal to) / GT(greater than)
    - optimize → Select based on extreme value:
        - ARGMAX(maximize) / ARGMIN(minimize)

    ## Output Specification
    - Output in single-line JSON format without any explanatory text
    - Must contain three fields (variables/constants/functions)
    - Empty fields represented with empty objects (e.g. "constants": {})
    """

for i, batch in enumerate(batches):
    print(f"batch: {i + 1}/{len(batches)}")
    messages_list = []
    for qa in batch:
        if qa['predicted_simple_question_type'] not in examples:
            qa['predicted_simple_question_type'] = 'simple'
        selected_examples = examples[qa['predicted_simple_question_type']]
        messages_list.append([{'role': 'system', 'content': prompt}] + selected_examples+ [{'role': 'user', 'content': concat_input(qa)}])
        
    resp_list = await LLM._batch_generate_async(messages_list)
    
    for j, qa in enumerate(batch):
        resp = resp_list[j]
        content = resp.choices[0].message.content
        try:
            content = re.search(r'\{[\s\S]*\}', content).group(0)
            template_and_replacements = json.loads(fix_json(content))
            template = fix_template(template_and_replacements['template'])
            replacements = template_and_replacements['replacements']
            qa['predicted_template'] = template
            qa['predicted_replacements'] = replacements
        except:
            qa['predicted_template'] = ''
            qa['predicted_replacements'] = ''

    if (i + 1) % 5 == 0:
        json.dump(test_data, open(f'output/{exp_name}/prediction.json', 'w'), indent=2)


[('sk-54964e5c3b8c4998a74f7d3e35b618ac', 'https://api.deepseek.com', 'deepseek-chat'), ('sk-785362add9834d1da5907f4621dc90a8', 'https://api.deepseek.com', 'deepseek-chat')]
batch: 1/190


100%|██████████| 20/20 [00:17<00:00,  1.14it/s]


batch: 2/190


100%|██████████| 20/20 [00:22<00:00,  1.14s/it]


batch: 3/190


...

100%|██████████| 20/20 [00:27<00:00,  1.35s/it]


batch: 160/190


100%|██████████| 20/20 [00:21<00:00,  1.07s/it]


batch: 161/190


 90%|█████████ | 18/20 [00:19<00:01,  1.30it/s]

In [ ]:
from utils.parse_expr import expression_to_sparql
from retriever.semantic_retriever import SemanticRetriever

entity_retriever = SemanticRetriever('entity')
relation_retriever = SemanticRetriever('relation')
type_retriever = SemanticRetriever('type')

def sub_fn_to_mid(expression):
    func_list = ['R', 'JOIN', 'AND', 'OR', 'DIFF', 'VALUES', 'DISTINCT', 'COUNT', 'GROUP_COUNT', 'GROUP_SUM', 'LT', 'LE', 'EQ', 'GE', 'GT', 'ARGMIN', 'ARGMAX', 'ALL', 'IS_TRUE']
    seg_list = expression.split()
    last_token, second_last_token = '', ''
    for i in range(len(seg_list)):
        token = seg_list[i].strip(')(')
        if token not in func_list and not token.isdigit():
            if last_token in ['R', 'JOIN'] or second_last_token == 'IS_TRUE':
                retriever = relation_retriever
            elif last_token == 'P31':
                retriever = type_retriever
            else:
                retriever = entity_retriever
            mid = retriever.semantic_search(token)[0][1]
            seg_list[i] = seg_list[i].replace(token, mid)
        second_last_token = last_token
        last_token = token
    new_expression = ' '.join(seg_list)
    return new_expression

In [ ]:
# import json
# exp_name = 'dev_each_type_50'
# test_data = test_data = json.load(open(f'output/{exp_name}/prediction.json', 'r'))

for qa in test_data:
    template = qa['predicted_template']
    replacements = qa['predicted_replacements']
    try:
        assert(template)
        for function in replacements['functions']:
            template = template.replace(function, replacements['functions'][function])
        if replacements['constants']:
            template = template.replace('number', str(replacements['constants']['number']))

        s_expression = template
        s_expression_fn = template
        for var in sorted(replacements['variables'], reverse=True):
            core_fn = replacements['variables'][var]
            if not is_close(core_fn):
                core_fn = fix_core(core_fn)
            if core_fn in qa['calibrated_cores_fn']:
                index = qa['calibrated_cores_fn'].index(core_fn)
                core = qa['calibrated_cores'][index]
            else:
                core = sub_fn_to_mid(core_fn)
            s_expression = s_expression.replace(var, core, 1)
            s_expression_fn = s_expression_fn.replace(var, core_fn, 1)
        qa['predicted_s_expression'] = s_expression
        qa['predicted_s_expression_fn'] = s_expression_fn
    except:
        qa['predicted_s_expression_fn'] = ''
        qa['predicted_s_expression'] = ''

In [ ]:
for qa in test_data:
    qa['actions'] = expression_to_sparql(qa['predicted_s_expression'])

json.dump(test_data, open(f'output/{exp_name}/prediction.json', 'w'), indent=2)